In [17]:
# Importing packages
import pandas as pd
import discogs_client
import re
import time
import requests
import json
import os
import concurrent.futures
from dotenv import load_dotenv

# Discogs

In [18]:
# Client
load_dotenv()

discogs = discogs_client.Client('Discogs_to_Notion', user_token=os.getenv('discogs_token'))
me = discogs.identity()

## Release ids of Discogs wantlist and collection

In [19]:
# Discogs wantlist
wantlist = me.wantlist
wantlist_ids = []
for item in wantlist:
    print(item)
    wantlist_ids.append(item.id)

<WantlistItem 13835553 'Ventura'>
<WantlistItem 13518819 'Not Waving, But Drowning'>
<WantlistItem 20846650 'Day/Night'>
<WantlistItem 4570366 'Random Access Memories'>
<WantlistItem 12448883 'Kingdoms In Colour'>
<WantlistItem 24470483 'No Sleep in LA'>
<WantlistItem 15511221 'Mordechai'>
<WantlistItem 24861332 'Hugo'>
<WantlistItem 7252111 'Currents'>
<WantlistItem 21709576 'Fragments'>
<WantlistItem 10435568 'Take a Rest'>
<WantlistItem 8827787 'Malibu'>
<WantlistItem 15461907 'Bubba'>
<WantlistItem 7557957 'To Pimp A Butterfly'>
<WantlistItem 21483991 'Can‘t Buy The Mood'>
<WantlistItem 7033756 'Walk Dance Talk Sing'>
<WantlistItem 15389727 'Circles'>
<WantlistItem 5877431 'Jungle'>


In [20]:
# Discogs collection
collection = me.collection_folders[0].releases
collection_ids = []
for item in collection:
    print(item)
    # Add release ids to list
    collection_ids.append(item.id)

<CollectionItemInstance 13113693 'Oxnard'>
<CollectionItemInstance 20095591 'Sometimes I Might Be Introvert'>
<CollectionItemInstance 12646784 'Parcels'>
<CollectionItemInstance 15222265 'Live Vol.1'>
<CollectionItemInstance 1183731 'Alligator'>
<CollectionItemInstance 14791710 'Suddenly'>
<CollectionItemInstance 19292665 'Cyclorama'>
<CollectionItemInstance 17526067 'Terra Firma'>
<CollectionItemInstance 13374359 'Significant Changes'>
<CollectionItemInstance 17030532 'Isles'>
<CollectionItemInstance 25286800 'Capricorn Sun'>
<CollectionItemInstance 24356900 'Mr. Morale & The Big Steppers'>
<CollectionItemInstance 17593180 'Sand'>
<CollectionItemInstance 4618675 'Settle'>
<CollectionItemInstance 12009384 'Wide Awake!'>
<CollectionItemInstance 14380475 'Are We Still Dreaming?'>
<CollectionItemInstance 7251550 'Venice'>
<CollectionItemInstance 26294105 'Remember'>
<CollectionItemInstance 2879 'Discovery'>
<CollectionItemInstance 14775292 'Cut The Wire '>
<CollectionItemInstance 12509101

## Get album info from Discogs

In [21]:
def get_album_info(release_ids, tag):
    # Create empty pandas dataframe with desired columns
    df = pd.DataFrame(columns=['album', 'artist', 'url', 'image'])

    def get_release_info(release_id):
        try:
            release = discogs.release(release_id)                                           # initiate release object using id
            album_title = release.title                                                     # album title
            artist_name = re.sub(r'\([^()]*\)', '', release.artists[0].name)                # artist name
            album_url = release.url                                                         # url
            album_image = release.images[0].get('uri')                                      # image

            # Add row to dataframe with album information
            df.loc[len(df)] = [album_title, artist_name, album_url, album_image]          

            return album_title, artist_name, album_url, album_image
        except:
            return None

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit requests for each release_id
        futures = [executor.submit(get_release_info, release_id) for release_id in release_ids]

        # Retrieve results as they become available
        for future in concurrent.futures.as_completed(futures):
            result = future.result()

            if result is not None:
                album_title, artist_name, album_url, album_image = result

                # Print to see progress
                print(album_title)
    
    tag_dict = {'wish': 'wish', 'collection': 'collection'}
    df['tags'] = tag_dict.get(tag, None)
                                    
    return df

In [22]:
discogs_collection = get_album_info(collection_ids, 'collection')
discogs_collection.head()

Oxnard
Parcels
Sometimes I Might Be Introvert
Alligator
Cyclorama
Terra Firma
Suddenly
Live Vol.1
Capricorn Sun
Isles
Significant Changes
Are We Still Dreaming?
Settle
Sand
Wide Awake!
Cut The Wire 
Mr. Morale & The Big Steppers
Remember
Venice
Discovery
For Ever
99.9%


,album,artist,url,image,tags
0,Oxnard,Anderson .Paak,https://www.discogs.com/release/13113693-Ander...,https://i.discogs.com/R9LcC7Y9Ij2vABwBhEGUtUvo...,collection
1,Parcels,Parcels,https://www.discogs.com/release/12646784-Parce...,https://i.discogs.com/9zhDlpy6cjuZw5RQV9ajAZwh...,collection
2,Sometimes I Might Be Introvert,Little Simz,https://www.discogs.com/release/20095591-Littl...,https://i.discogs.com/0SRVBvcx_63Td-IR9dHpVcEp...,collection
3,Alligator,The National,https://www.discogs.com/release/1183731-The-Na...,https://i.discogs.com/bWcIWn90Wb2iByGmmVIw-GrS...,collection
4,Cyclorama,Polo & Pan,https://www.discogs.com/release/19292665-Polo-...,https://i.discogs.com/32KU1dwGGqzA82WtsgJ_CAVT...,collection


In [23]:
discogs_wantlist = get_album_info(wantlist_ids, 'wish')
discogs_wantlist.head()

Ventura
Day/Night
No Sleep in LA
Kingdoms In Colour
Hugo
Mordechai
Not Waving, But Drowning
Currents
Malibu
Walk Dance Talk Sing
Take a Rest
Bubba
Jungle
Fragments
Can‘t Buy The Mood
Circles
To Pimp A Butterfly
Random Access Memories


,album,artist,url,image,tags
0,Ventura,Anderson .Paak,https://www.discogs.com/release/13835553-Ander...,https://i.discogs.com/nRnMfcb_WpE80LhuwiAwPZQp...,wish
1,Day/Night,Parcels,https://www.discogs.com/release/20846650-Parce...,https://i.discogs.com/c87RZSedRpAzG3GUpvgjj52F...,wish
2,No Sleep in LA,Blackwave,https://www.discogs.com/release/24470483-Black...,https://i.discogs.com/sHAuC85M2hOF2zvDYx5UgSQt...,wish
3,Kingdoms In Colour,Maribou State,https://www.discogs.com/release/12448883-Marib...,https://i.discogs.com/izKlrBtyqrtHt80RP4CJvkbC...,wish
4,Hugo,Loyle Carner,https://www.discogs.com/release/24861332-Loyle...,https://i.discogs.com/sLP9Abw1ERF3n6A1sfsJt-J-...,wish


# Notion

In [24]:
# Initialization
token = os.getenv('notion_token')
database_id = '1afa86cc349c402ab660a19466400390'
headers = {
    'Authorization': 'Bearer ' + token,
    'Content-type': 'application/json',
    'Notion-Version': '2022-06-28'
}

## Get pages info (read)

In [25]:
def get_pages():
    read_url = f'https://api.notion.com/v1/databases/{database_id}/query'

    response = requests.post(read_url, headers=headers)

    data = response.json()
    with open('db.json', 'w', encoding='utf8') as f:    
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    results = data['results']
    return results

In [26]:
pages = get_pages()

def get_pages_info(pages):

    albums = []
    artists = []
    urls = []
    tags = []
    page_ids = []
    images = []

    for page in pages:
       
        page_id = page['id']
        props = page['properties']
        album_title = props['Album']['title'][0]['text']['content']
        artist = props['Artist']['multi_select'][0]['name']
        album_url = props['URL']['url']
        tag = props['Tags']['multi_select'][0]['name']
        image = props['Album cover']['files'][0]['external']['url']

        # Append info to lists
        albums.append(album_title)
        artists.append(artist)
        urls.append(album_url)
        tags.append(tag)
        images.append(image)
        page_ids.append(page_id)

    df = pd.DataFrame(zip(albums, artists, urls, tags, images, page_ids),
                                    columns = ['album', 'artist', 'url', 'tags', 'image', 'page_id'])
    
    return df

In [27]:
notion_pages = get_pages_info(pages)
notion_pages.head()

,album,artist,url,tags,image,page_id
0,Jungle,Jungle,https://www.discogs.com/release/5877431-Jungle...,wish,https://i.discogs.com/U_5IUuBKZ8mv3NUBSNfDxtTm...,5666edca-77e9-4c79-9c46-92b893e273eb
1,Circles,Mac Miller,https://www.discogs.com/release/15389727-Mac-M...,wish,https://i.discogs.com/dtBh-cZSW43w-cItbzZdtNxt...,8fa43b14-ac98-4175-b773-9040d475cd14
2,Walk Dance Talk Sing,Crazy Penis,https://www.discogs.com/release/7033756-Crazy-...,wish,https://i.discogs.com/3yFASOTnu6OQ5k1XfLy-XqD7...,b585baf6-ef62-4983-b756-ab30242f918c
3,Can‘t Buy The Mood,Tora,https://www.discogs.com/release/21483991-Tora-...,wish,https://i.discogs.com/MMbOJ1J_64X-QoBovqeviUTL...,5cab08ca-10d0-4276-b511-496aa7ff8366
4,To Pimp A Butterfly,Kendrick Lamar,https://www.discogs.com/release/7557957-Kendri...,wish,https://i.discogs.com/mzZN2JyPeiMrz8YnZd04bKif...,ed27d488-c29e-4c9b-877b-e73044578f60


## Delete old pages

In [28]:
# Function to delete multiple pages at once using multi-threading
def delete_pages(page_ids: list):

    # Function for the deletion of one single page
    def delete_page(page_id: str):
        url = f"https://api.notion.com/v1/pages/{page_id}"

        payload = {"archived": True}

        res = requests.patch(url, json=payload, headers=headers)
        return page_id, res

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(delete_page, page_id) for page_id in page_ids]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            # Print response code after each page deletion
            print(result)

In [29]:
# Get a list of page IDs to delete
page_ids = notion_pages['page_id'].tolist()

# Delete the pages using multi-threading
delete_pages(page_ids)

('b585baf6-ef62-4983-b756-ab30242f918c', <Response [200]>)
('5cab08ca-10d0-4276-b511-496aa7ff8366', <Response [200]>)
('40464f9f-4fee-4dfe-b023-5e2af7a5a43b', <Response [200]>)
('0e72b1d8-2062-420e-8371-e57b65d90c98', <Response [200]>)
('f693801c-1208-4675-babb-c62e17b63b93', <Response [200]>)
('9bef2c89-2fed-4c7e-826e-63fd34f966d4', <Response [200]>)
('95067f7e-1261-4399-92d4-c7384440a1a0', <Response [200]>)
('fe1534bc-331e-47b4-ab57-7d7f7f7fbb7e', <Response [200]>)
('d4218f98-23e1-4ca0-901a-16a2c82edb40', <Response [200]>)
('5666edca-77e9-4c79-9c46-92b893e273eb', <Response [200]>)
('444bd8b7-9d50-4f80-8c0b-ca130904fd1f', <Response [200]>)
('d5244895-e9e9-46e1-b9c4-2ab7404b995e', <Response [200]>)
('ed27d488-c29e-4c9b-877b-e73044578f60', <Response [200]>)
('ba7240b2-b427-489a-a483-ba50506bbc3e', <Response [200]>)
('6fbed134-2065-442d-9de7-46a56714476a', <Response [200]>)
('dfd123e1-9673-4c94-b557-85480d148d2e', <Response [200]>)
('16b50f8f-0421-41bc-9297-ba5071dc69a8', <Response [200]

## Create new pages using Discogs info

In [30]:
# Function to create multiple pages at once using multi-threading
def create_pages(data_list: list):

    # Function for the creation of one single page
    def create_page(data: dict):
        create_url = 'https://api.notion.com/v1/pages'

        payload = {'parent': {'database_id': database_id}, 'properties': data}

        res = requests.post(create_url, headers=headers, json=payload)
        
        # Retrieve album name
        album = data.get('Album')
        album = album['title'][0]['text']['content']

        # Return album name and corresponding response code
        return album, res

    # Multi-threading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(create_page, data) for data in data_list]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            # Print response code after creating each page
            print(result)

In [31]:
def setup_data(df):
    # Define tag colours
    colours = {'wish': 'purple', 'collection': 'green'}

    # Define empty list for adding data dicts
    data_list = []

    for index, row in df.iterrows():
        # Extract data from df
        album = row['album']
        artist = row['artist']
        url = row['url']
        tag = row['tags']
        image = row['image']
        color = colours.get(tag, 'Blue')          # Define tag color based on tag value

        # Setting up data for new page
        data = {
            'Album': {'title': [{'text': {'content': album}}]},
            'Artist': {'multi_select': [{'name': artist}]},
            'URL': {'url': url},
            'Tags': {'multi_select': [{'name': tag, 'color': color}]},
            'Album cover': {'files': [{'name': 'image', 'type': 'external', 'external': {'url': image}}]}
            }

        data_list.append(data)
    
    # Create pages using multi-threading
    create_pages(data_list)

In [32]:
# Setup data and create new pages
setup_data(discogs_collection)
setup_data(discogs_wantlist)

('Parcels', <Response [200]>)
('Terra Firma', <Response [200]>)
('Suddenly', <Response [200]>)
('Sand', <Response [200]>)
('Wide Awake!', <Response [200]>)
('Cut The Wire ', <Response [200]>)
('Settle', <Response [200]>)
('Cyclorama', <Response [200]>)
('Live Vol.1', <Response [200]>)
('Significant Changes', <Response [200]>)
('Capricorn Sun', <Response [200]>)
('Discovery', <Response [200]>)
('Oxnard', <Response [200]>)
('Isles', <Response [200]>)
('Remember', <Response [200]>)
('Sometimes I Might Be Introvert', <Response [200]>)
('Mr. Morale & The Big Steppers', <Response [200]>)
('Are We Still Dreaming?', <Response [200]>)
('Alligator', <Response [200]>)
('For Ever', <Response [200]>)
('99.9%', <Response [200]>)
('Venice', <Response [200]>)
('Bubba', <Response [200]>)
('Hugo', <Response [200]>)
('Mordechai', <Response [200]>)
('Kingdoms In Colour', <Response [200]>)
('Take a Rest', <Response [200]>)
('Not Waving, But Drowning', <Response [200]>)
('No Sleep in LA', <Response [200]>)
